
# Weight Editing 101 — From First Weights to Direct Editing (PyTorch)

이 노트북은 **가중치(Weights)가 어떻게 생성되고, 어떤 파일 포맷으로 저장/로드하며, 어떻게 직접 수정(Weight Editing)하는지**를 단계별로 보여줍니다.  
또한 **파인튜닝(Fine-tuning)**과의 차이를 코드 레벨에서 비교하고, 초보자가 실무에서 주의할 점을 체크리스트로 정리합니다.

> 실행 환경 메모: `safetensors`, `onnx`, `onnxruntime`이 설치되어 있지 않을 수도 있습니다.  
> 이 노트북은 해당 라이브러리가 없으면 **자동으로 해당 섹션을 건너뛰고** 계속 진행합니다.


## 0) 준비

In [ ]:

import sys, platform, importlib
print(f"Python: {sys.version}")
print(f"Platform: {platform.platform()}")

# 필수: PyTorch
try:
    import torch, torch.nn as nn, torch.optim as optim
    TORCH_OK = True
    print("PyTorch import: OK")
except Exception as e:
    TORCH_OK = False
    print("PyTorch import: FAILED:", e)

# 선택: safetensors
try:
    from safetensors.torch import save_file as safetensors_save_file, load_file as safetensors_load_file
    SAFETENSORS_OK = True
    print("safetensors import: OK")
except Exception as e:
    SAFETENSORS_OK = False
    print("safetensors import: not available")

# 선택: ONNX & onnxruntime
try:
    import onnx
    import torch.onnx as to_onnx
    ONNX_OK = True
    print("onnx import: OK")
except Exception as e:
    ONNX_OK = False
    print("onnx import: not available")

try:
    import onnxruntime as ort
    ONNXRUNTIME_OK = True
    print("onnxruntime import: OK")
except Exception as e:
    ONNXRUNTIME_OK = False
    print("onnxruntime import: not available")

if TORCH_OK:
    torch.manual_seed(42)



## 1) “가중치가 만들어지는” 가장 기초

`nn.Linear` 같은 레이어는 생성 시 **무작위 초기화**로 가중치/바이어스를 만듭니다.


In [ ]:

if not TORCH_OK:
    raise RuntimeError("PyTorch가 필요합니다. 위 셀의 PyTorch import가 실패했습니다.")

import torch
import torch.nn as nn

# 입력 4 → 출력 3
layer = nn.Linear(in_features=4, out_features=3, bias=True)

print("W shape:", layer.weight.shape)  # [3, 4]
print("b shape:", layer.bias.shape)    # [3]

x = torch.randn(1, 4)
y = layer(x)
print("sample input:", x)
print("y:", y)



## 2) 가중치 파일 포맷과 저장/불러오기

### 2.1 PyTorch 표준 (`.pt`/`.pth`; Pickle 기반)
- 일반적으로 **`state_dict`**만 저장/로드하는 것이 권장됩니다.
- Pickle 기반이라 **신뢰할 수 있는 파일만 로드**하세요.


In [ ]:

import torch

# 저장 (state_dict 권장)
torch.save(layer.state_dict(), "/mnt/data/linear_state.pt")
print("Saved: /mnt/data/linear_state.pt")

# 동일 구조의 새 레이어에 로드
layer2 = nn.Linear(4, 3, bias=True)
state_loaded = torch.load("/mnt/data/linear_state.pt", map_location="cpu")
layer2.load_state_dict(state_loaded)

# 일치 여부 대략 확인
with torch.no_grad():
    diff = (layer.weight - layer2.weight).abs().max().item()
print("Max weight diff between layer & layer2:", diff)



### 2.2 `safetensors` (보안/속도/무시리얼라이즈 장점)
- pickle이 아니므로 **안전한 배포/공유**에 유리합니다.
- 없으면 자동으로 건너뜁니다.


In [ ]:

if SAFETENSORS_OK:
    safetensors_save_file(layer.state_dict(), "/mnt/data/linear_state.safetensors")
    print("Saved: /mnt/data/linear_state.safetensors")

    st = safetensors_load_file("/mnt/data/linear_state.safetensors")
    layer3 = nn.Linear(4, 3, bias=True)
    layer3.load_state_dict(st)
    with torch.no_grad():
        diff = (layer.weight - layer3.weight).abs().max().item()
    print("Max weight diff between layer & layer3:", diff)
else:
    print("safetensors unavailable — skipping this section.")



### 2.3 ONNX (모델 구조 + 가중치를 그래프로 내보내기)
- `onnx`와 `onnxruntime`이 모두 있을 때 예제를 실행합니다.
- 보통은 **PyTorch에서 수정 → ONNX로 재내보내기**가 현실적입니다.


In [ ]:

if ONNX_OK:
    import torch.onnx as to_onnx
    dummy = torch.randn(1, 4)
    to_onnx.export(
        layer, dummy, "/mnt/data/linear.onnx",
        input_names=["x"], output_names=["y"], opset_version=17
    )
    print("Exported: /mnt/data/linear.onnx")

    if ONNXRUNTIME_OK:
        sess = ort.InferenceSession("/mnt/data/linear.onnx", providers=["CPUExecutionProvider"])
        out = sess.run(["y"], {"x": dummy.numpy()})[0]
        print("onnxruntime output shape:", out.shape)
    else:
        print("onnxruntime not available — cannot run inference here.")
else:
    print("onnx unavailable — skipping ONNX export.")



## 3) 가중치를 **직접** 수정 (Weight Editing)

> 옵티마이저/학습 없이 **텐서 값을 직접 변경**하는 행위입니다.


In [ ]:

# 3.1 선형 레이어의 특정 행/열 덮어쓰기
with torch.no_grad():
    # 첫 번째 출력 뉴런의 가중치를 모두 0으로
    layer.weight[0, :] = 0.0
    # 바이어스도 원하는 값으로
    layer.bias[0] = 1.0

print("Edited weight[0]:", layer.weight[0])
print("Edited bias[0]:", layer.bias[0])

# 3.2 임베딩에서 특정 토큰 벡터 수정 (LLM 임베딩 편집의 축소판)
emb = nn.Embedding(num_embeddings=10, embedding_dim=5)
token_id = 3
print("원래 임베딩(토큰 3):", emb.weight[token_id])

with torch.no_grad():
    new_vec = torch.tensor([1.0, -1.0, 0.5, 0.5, 0.0])
    emb.weight[token_id] = new_vec

print("수정 후 임베딩(토큰 3):", emb.weight[token_id])


In [ ]:

# 3.3 저랭크 업데이트 아이디어 흉내(LoRA 느낌)
W = layer.weight.data           # [out, in]
rank = 1
A = torch.randn(W.size(0), rank)
B = torch.randn(W.size(1), rank)
delta = A @ B.t()

with torch.no_grad():
    layer.weight += 0.01 * delta

print("Applied small low-rank delta to layer.weight")



## 4) “가중치 수정” vs “파인튜닝(학습)” 비교 (코드 관점)
- **편집**: 목적함수 없이 즉각적 변경 — 빠르지만 예측 불가.
- **파인튜닝**: 손실함수/데이터로 체계적 업데이트 — 안정적·재현 가능.


In [ ]:

# 4.1 (편집) 수동 변경
with torch.no_grad():
    noise = 0.001 * torch.randn_like(layer.weight)
    layer.weight += noise
print("Weight edited with small random noise (no optimizer)")


In [ ]:

# 4.2 (학습) 미니 배치 파인튜닝
model = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 3))
opt = optim.Adam(model.parameters(), lr=1e-2)
loss_fn = torch.nn.MSELoss()

x = torch.randn(32, 4)
target = torch.randn(32, 3)

model.train()
for step in range(20):
    opt.zero_grad()
    pred = model(x)
    loss = loss_fn(pred, target)
    loss.backward()
    opt.step()
    if step % 5 == 0:
        print(f"step {step:02d} | loss {loss.item():.4f}")



## 5) 저장된 가중치 파일을 **편집**해서 다시 쓰기

### 5.1 PyTorch `state_dict` 편집
- 키 이름을 확인하고, 원하는 텐서를 수정한 뒤 다시 저장합니다.


In [ ]:

state = torch.load("/mnt/data/linear_state.pt", map_location="cpu")
print("Keys in state_dict:", list(state.keys()))

# 예: weight의 첫 행을 0으로
with torch.no_grad():
    state["weight"][0] = 0.0

torch.save(state, "/mnt/data/linear_state_edited.pt")
print("Saved edited state_dict to: /mnt/data/linear_state_edited.pt")

# 주입 테스트
layer_edited = nn.Linear(4, 3, bias=True)
layer_edited.load_state_dict(torch.load("/mnt/data/linear_state_edited.pt", map_location="cpu"))
print("Loaded edited state into new layer.")



### 5.2 `safetensors` 편집
- 텐서만 담겨 있으므로 키와 shape을 정확히 알고 있어야 합니다.


In [ ]:

if SAFETENSORS_OK:
    st = safetensors_load_file("/mnt/data/linear_state.safetensors")
    print("Keys:", list(st.keys()))
    with torch.no_grad():
        st["weight"][1] += 0.123  # 두 번째 행에 상수 더하기
    safetensors_save_file(st, "/mnt/data/linear_state_edited.safetensors")
    print("Saved: /mnt/data/linear_state_edited.safetensors")

    layer_edited2 = nn.Linear(4, 3, bias=True)
    layer_edited2.load_state_dict(safetensors_load_file("/mnt/data/linear_state_edited.safetensors"))
    print("Loaded edited safetensors into new layer.")
else:
    print("safetensors unavailable — skipping edit example.")



## 6) ONNX 가중치 직접 편집에 관하여

- ONNX는 가중치를 **initializer**로 그래프에 포함합니다. 직접 편집은 가능하지만 번거롭습니다.
- 일반적으로 **PyTorch에서 편집 → ONNX로 재-export**가 현실적인 워크플로입니다.



## 7) 실무 체크리스트 (초보자 필수)

- **state_dict 저장 권장**: `torch.save(model.state_dict(), path)`  
  전체 모델 저장은 코드 의존·보안 이슈로 지양.
- **Pickle 보안**: `.pt/.pth`는 pickle 기반 → **신뢰 가능한 파일만 로드**. 배포/교차 조직 공유에는 **safetensors 권장**.
- **`torch.no_grad()`**: 수동 편집 시 그래프 오염/메모리 낭비 방지.
- **정밀도(dtype)**: fp32/fp16/bf16/int8/int4 등 경량화는 정확도 트레이드오프. 벤치마크 필수.
- **`model.eval()`**: 추론 시 드롭아웃/BatchNorm 고정.
- **키/shape 검증**: `load_state_dict(..., strict=True)`로 안전하게. 불가피할 때만 `strict=False`.
- **버전 고정**: 프레임워크/커스텀 코드 버전 차이로 로드 실패 가능 → `requirements.txt` 및 커밋 해시 관리.
- **대형 모델 편집 리스크**: 작은 편집도 광범위한 부작용 가능 → **카나리아 테스트/가드레일/회귀 테스트** 필수.



## 8) 요약

- **가중치 수정(Weight Editing)**: 학습 없이 텐서 값을 직접 바꾸는 것 → 빠르지만 예측 불가.
- **파인튜닝**: 데이터/손실함수로 체계적으로 업데이트 → 안정적·재현 가능.
- **RAG**: 가중치를 건드리지 않고 검색 결과를 프롬프트에 주입 → 최신성/소유 데이터 반영에 유리.
- 파일 포맷은 **state_dict(PyTorch)**가 기본, 공유/배포에는 **safetensors**가 안전, 배포용 런타임에는 **ONNX**가 유용.
